In [1]:
from os import listdir
from os.path import isfile, join

import pickle
import numpy as np
import scipy as sp
import pandas as pd

from tensorflow.keras.models import load_model
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.models import Model

from sklearn.neighbors import DistanceMetric

from benchmark.cfg import OHConverter

from constants.var_types import VAR_TYPES

In [2]:
import os
import tensorflow as tf
tf.config.set_visible_devices([], 'GPU')

os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


In [3]:
# Custom Realistic Scores

In [4]:
def internetadv_ratio_contraint(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        scaler = load_scaler(dsName)
        df_num_scaled = pd.DataFrame(scaler.inverse_transform(df_cf_found[['1555', '1556', '1557']]))

        output = ((df_num_scaled[1] / df_num_scaled[0]).map(lambda x: round(x, 2)) - df_num_scaled[2].map(
            lambda x: round(x, 2)) < 0.01).map(int)
        output.index = df_cf_found.index

        return output

    return np.nan

In [5]:
def pbc_multivariate_constraints(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        scaler = load_scaler(dsName)
        cf_scaled = pd.DataFrame(scaler.inverse_transform(df_cf_found))

        x1 = (cf_scaled[2].map(lambda x: round(x, 4)) == (cf_scaled[0] * cf_scaled[1]).map(lambda x: round(x, 4))).map(
            int)

        x2 = (cf_scaled[3].map(lambda x: round(x, 3)) - (cf_scaled[1] / cf_scaled[0]).map(
            lambda x: round(x, 3)) < 0.01).map(int)

        x3 = (cf_scaled[4].map(lambda x: round(x, 3)) - (cf_scaled[7] / cf_scaled[2]).map(
            lambda x: round(x, 3)) < 0.01).map(int)

        x4 = (cf_scaled[5].map(lambda x: round(x, 3)) - (cf_scaled[8] / cf_scaled[2]).map(
            lambda x: round(x, 3)) < 0.01).map(int)

        x5 = (cf_scaled[6].map(lambda x: round(x, 3)) - (cf_scaled[7] / cf_scaled[9]).map(
            lambda x: round(x, 3)) < 0.01).map(int)

        output = (x1 + x2 + x3 + x4 + x5).map(lambda x: x == 5).map(int)
        output.index = df_cf_found.index

        return output

    return np.nan

In [6]:
def bcw_radius_area_constraint(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        scaler = load_scaler(dsName)
        cf_scaled = pd.DataFrame(scaler.inverse_transform(df_cf_found))

        output = (cf_scaled[1] ** 2 * np.pi - cf_scaled[4] < cf_scaled[4] * 0.10).map(int)
        output.index = df_cf_found.index

        return output

    return np.nan

In [7]:
dict_functions = {
    'BCW': {'ruc': [], 'rmc': [bcw_radius_area_constraint]},
    'PBC': {'ruc': [], 'rmc': [pbc_multivariate_constraints]},
    'InternetAdv': {'ruc': [], 'rmc': [internetadv_ratio_contraint]}
}

In [8]:
# Verify if offset between the x and o classes is, at maximum 1
def tictactoe_constraint(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        if cat_feats:

            df_ohi = check_one_hot_integrity(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
            df_cb = check_binary_categorical(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)

            dfc_converted_array = []
            for idx in range(df_cf_found.shape[0]):
                if df_ohi.iloc[idx] == 1 and df_cb.iloc[idx] == 1:
                    dfc_converted_array.append(converter.convert(df_cf_found.iloc[int(idx):int(idx) + 1].to_numpy())[0])
                else:
                    dfc_converted_array.append(['x'] * df_cf_found.shape[1])
            dfc_converted = pd.DataFrame(dfc_converted_array)

            dict_ttt = {'x': 1, 'o': -1, 'b': 0}
            for col in list(dfc_converted.columns):
                dfc_converted[col] = dfc_converted[col].apply(lambda x: dict_ttt[x])

            output = dfc_converted.sum(axis=1).map(lambda x: int(x in [0, 1, -1]))
            output.index = df_cf_found.index

            return output

    return np.nan

In [9]:
# Realistic Scores

In [10]:
def check_outside_num_range(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    if df_cf_found.shape[0] > 0:
        if len(num_feats) > 0:
            scaler = load_scaler(dsName)
            scaled_ds = pd.DataFrame(scaler.inverse_transform(df_oh[num_feats]))

            num_feats_idx = []

            for n_f in num_feats:
                num_feats_idx.append(str(list(df_oh.columns).index(n_f)))
            scaled_cf = pd.DataFrame(scaler.inverse_transform(df_cf_found[num_feats_idx]))

            for i in range(scaled_ds.shape[1]):
                scaled_cf[i] = scaled_cf[i].map(lambda x: scaled_ds[i].min() <= x and x <= scaled_ds[i].max())

            output = scaled_cf.T.sum().map(lambda x: x == scaled_cf.shape[1]).map(int)
            output.index = df_cf_found.index

            return output
    return np.nan

In [11]:
def load_scaler(dsName):
    return pickle.load(open(f'./scalers/{dsName}.sc', 'rb'))

In [12]:
def verify_one_hot(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    oh_i = check_one_hot_integrity(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)
    oh_b = check_binary_categorical(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter)

    return (oh_i * oh_b).sum() / oh_b.shape[0]

In [13]:
def check_one_hot_integrity(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)
    bin_feats = converter.binary_cats

    if df_cf_found.shape[0] > 0:
        if len(cat_feats) > 0:
            cat_oh_feats = list(set(cat_feats) - set(bin_feats))

            if len(cat_oh_feats) > 0:
                array_cat_int = []
                for cat_oh_f in cat_oh_feats:
                    array_cat_int.append(
                        df_cf_found.iloc[:, list(converter.dict_feat_idx[cat_oh_f].values())].apply(lambda x: x > 0,
                                                                                                    axis=1).sum(
                            axis=1).map(lambda x: x == 1))

                df_cat_int = pd.concat(array_cat_int, axis=1)

                output = df_cat_int.sum(axis=1).map(lambda x: x == df_cat_int.shape[1]).map(int)
                output.index = df_cf_found.index

                return output

    return np.nan

In [14]:
def check_binary_categorical(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)

    if df_cf_found.shape[0] > 0:

        if len(cat_feats) > 0:
            num_feats_idx = [str(list(df_oh.columns).index(n_f)) for n_f in num_feats]

            cat_feats_idx = list(set(df_cf_found.columns) - set(num_feats_idx))

            factual_cat_cols = df_fc_found.loc[:, cat_feats_idx].apply(
                lambda x: x.map(lambda y: y not in [1.0, 0.0, 1, 0]), axis=1).T.sum()

            assert factual_cat_cols.sum() == 0

            output = df_cf_found.loc[:, cat_feats_idx].apply(lambda x: x.map(lambda y: y in [1.0, 0.0, 1, 0]),
                                                             axis=1).apply(sum, axis=1).map(
                lambda x: int(x == len(cat_feats_idx)))
            output.index = df_cf_found.index

            return output

    return np.nan

In [15]:
def get_bin_feats_idx(df_oh, converter):
    bin_cols = converter.binary_cats
    bin_feats = []
    for b_c in bin_cols:
        try:
            bin_feats.append(list(df_oh.columns).index(b_c))
        except ValueError:
            bin_feats.append(list(df_oh.columns).index('0'*(4-len(b_c))+b_c))
    return bin_feats

In [16]:
# General Scores

In [17]:
def md(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)

    if df_cf_found.shape[0] > 0:
        output_result = []
        for idx in range(df_cf_found.shape[0]):
            m_dis = sp.spatial.distance.mahalanobis(df_cf_found.iloc[idx].to_numpy(),
                                                    df_fc_found.drop(columns=['output']).iloc[idx].to_numpy(),
                                                    df_oh.drop(columns=['output']).cov().to_numpy())
            output_result.append(m_dis)

        out_array = output_result
        output_results = [np.nan] * df_cf.shape[0]
        for idx_result, idxFound in enumerate(list(df_fc_found.index)):
            output_results[idxFound] = out_array[idx_result]

        return output_results

    return [np.nan]*df_cf.shape[0]

In [18]:
# Mean can give misleading values because very high values on MAD
def madd(df_oh, df_cf, df_fc, cat_feats, num_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)

    if df_cf_found.shape[0] > 0:
        df_mad = df_cf_found.iloc[:0]

        df_oh_c = df_oh.copy()
        df_oh_c.columns = df_fc_found.columns
        num_feats_idx = [str(list(df_oh.columns).index(n_f)) for n_f in num_feats]

        cat_feats_idx = list(set(df_cf_found.columns) - set(num_feats_idx))

        mad_num = {}
        for n_feat_idx in num_feats_idx:
            # 1e-8 added to avoid 0 and, then, division by zero
            mad_num[n_feat_idx] = sp.stats.median_abs_deviation(df_oh_c[n_feat_idx]) + 1e-8

            df_mad[n_feat_idx] = abs(df_cf_found[n_feat_idx] - df_fc_found[n_feat_idx]) / mad_num[n_feat_idx]

        for c_feat_idx in cat_feats_idx:
            df_mad[c_feat_idx] = (df_cf_found[c_feat_idx] != df_fc_found[c_feat_idx]).map(int)

        output_result = [0]*df_cf.shape[0]
        if len(cat_feats_idx) > 0:
            add_output_result = df_mad[cat_feats_idx].mean(axis=1)
            for null_row in list(set([*range(len(output_result))])-set(df_mad.index)):
                add_output_result.loc[null_row] = np.nan
            add_output_result = add_output_result.sort_index()
            output_result = np.add(output_result, add_output_result.tolist())
        if len(num_feats_idx) > 0:
            add_output_result = df_mad[num_feats_idx].mean(axis=1)
            for null_row in list(set([*range(len(output_result))])-set(df_mad.index)):
                add_output_result.loc[null_row] = np.nan
            add_output_result = add_output_result.sort_index()
            output_result = np.add(output_result, add_output_result.tolist())

        out_array = output_result.tolist()
        output_results = [np.nan] * df_cf.shape[0]
        for idx_result, idxFound in enumerate(list(df_fc_found.index)):
            output_results[idxFound] = out_array[idx_result]

        return output_results

    return [np.nan]*df_cf.shape[0]

In [19]:
def get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter):
    df_fc_pro = get_df_fc_pro(df_cf, df_fc, cat_feats, converter)
    df_cf_c, df_fc_pro_c = get_correct(df_cf, df_fc_pro, model)

    return df_cf_c, df_fc_pro_c

In [20]:
def get_correct(df_cf, df_fc, model):
    df_cf_found, df_fc_found = remove_na(df_cf, df_fc)
    if df_cf_found.shape[0] > 0:
        labels_cf = model.predict(df_cf_found.to_numpy()).round()
        return df_cf_found[labels_cf.reshape(-1, 1) != df_fc_found['output'].to_numpy().reshape(-1, 1)], df_fc_found[
            labels_cf.reshape(-1, 1) != df_fc_found['output'].to_numpy().reshape(-1, 1)]

    return df_cf_found.iloc[:0], df_fc.iloc[:0]

In [21]:
def validity_total(df_cf, df_fc, model):
    df_cf_found, df_fc_found = remove_na(df_cf, df_fc)
    if df_cf_found.shape[0] > 0:

        out_array = (model.predict(df_cf_found.to_numpy()).round().reshape(1,-1)!=df_fc_found['output'].to_numpy())[0].tolist()
        output_results = [np.nan]*df_cf.shape[0]
        for idx_result, idxFound in enumerate(list(df_fc_found.index)):
            output_results[idxFound] = out_array[idx_result]

        return output_results
    return [False]*df_cf.shape[0]

In [22]:
def validity_found(df_cf, df_fc, model):
    df_cf_found, df_fc_found = remove_na(df_cf, df_fc)
    if df_cf_found.shape[0] > 0:

        out_array = (model.predict(df_cf_found.to_numpy()).round().reshape(1,-1)!=df_fc_found['output'].to_numpy())[0].tolist()
        output_results = [np.nan] * df_cf.shape[0]
        for idx_result, idxFound in enumerate(list(df_fc_found.index)):
            output_results[idxFound] = out_array[idx_result]

        return output_results
    return [False]*df_cf.shape[0]

In [23]:
def remove_na(df_cf, df_fc):
    return df_cf.dropna(), df_fc[0==df_cf.isna().sum(axis=1)]

In [24]:
def get_df_fc_pro(df_cf, df_fc, cat_feats, converter):
    array_df_fc_pro = []
    if len(cat_feats) > 0:
        for idx, row in df_fc.iterrows():
            row_fc_pro = converter.convert_to_oh(pd.DataFrame(row).T.drop(columns=['output']).to_numpy().tolist()[0])
            array_df_fc_pro.append(row_fc_pro)
        df_fc_pro = pd.DataFrame(array_df_fc_pro)
        df_fc_pro['output'] = df_fc['output']
    else:
        df_fc_pro = df_fc.copy()

    df_fc_pro.columns = list(df_cf.columns) + ['output']

    return df_fc_pro

In [25]:
def sparsity(df_cf, df_fc, cat_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)

    if df_cf_found.shape[0] > 0:
        df_fc_found = df_fc_found.copy().drop(columns=['output'])

        scores = []
        for i in range(df_cf_found.shape[0]):
            scores.append((df_cf_found.iloc[i].round(4) == df_fc_found.iloc[i].round(4)).sum() / df_cf_found.shape[1])

        out_array = scores
        output_results = [np.nan] * df_cf.shape[0]
        for idx_result, idxFound in enumerate(list(df_fc_found.index)):
            output_results[idxFound] = out_array[idx_result]

        return output_results

    return [np.nan]*df_cf.shape[0]

In [26]:
def l2(df_cf, df_fc, cat_feats, model, converter):
    df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, model, converter)

    if df_cf_found.shape[0] > 0:
        df_fc_found = df_fc_found.copy().drop(columns=['output'])
        scores = []
        for i in range(df_cf_found.shape[0]):
            scores.append(np.linalg.norm(
                df_cf_found.iloc[i].round(4).to_numpy().reshape(-1, 1) -
                df_fc_found.iloc[i].round(4).to_numpy().reshape(-1, 1)))

        out_array = scores
        output_results = [np.nan] * df_cf.shape[0]
        for idx_result, idxFound in enumerate(list(df_fc_found.index)):
            output_results[idxFound] = out_array[idx_result]

        return output_results

    return [np.nan]*df_cf.shape[0]

In [27]:
cat_ds = []
num_ds = []
mix_ds = []

for dsName, dsData in VAR_TYPES.items():

    if len(dsData['categorical']) > 0 and len(dsData['numerical']):
        mix_ds.append(dsName)
    elif len(dsData['categorical']) > 0:
        cat_ds.append(dsName)
    else:
        num_ds.append(dsName)

In [28]:
onlyfiles = [f for f in listdir('./cfoutput') if isfile(join('./cfoutput', f))]

In [29]:
def _nn_ohe(input_shape, hidden_layers_ws, output_number):
    x_in = Input(shape=(input_shape,))
    x = Dense(hidden_layers_ws, activation='relu')(x_in)
    x_out = Dense(2, activation='softmax')(x)
    if output_number == 1:
        x_bin = Dense(1, activation='linear')(x_out)
        nn = Model(inputs=x_in, outputs=x_bin)
    if output_number == 2:
        nn = Model(inputs=x_in, outputs=x_out)

    nn.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

    return nn

In [30]:
dict_results = {}
for file in onlyfiles:
    if file.split('_')[0] != 'TIME':
        n = file.split('_')[0]
        if n == 'STABILITY':
            continue
        dsName = file.split('_')[1]
        framework = file.split('_')[2].split('.')[0]

        cat_feats = VAR_TYPES[dsName]['categorical']
        num_feats = VAR_TYPES[dsName]['numerical']

        model_keras = load_model(f'./models/{dsName}.h5')

        # Get architecture info and pass to our NN architecture
        input_shape = model_keras.get_weights()[0].shape[0]
        hidden_layers_ws = model_keras.get_weights()[0].shape[1]

        adapted_nn = _nn_ohe(input_shape, hidden_layers_ws, 1)
        adapted_nn.build(input_shape=input_shape)

        adapted_nn.layers[1].set_weights(model_keras.layers[0].get_weights())
        adapted_nn.layers[2].set_weights(model_keras.layers[1].get_weights())

        if n == 1:
            # If the tested class is the minority, revert output order
            second_layer_w = []
            for w_row in model_keras.layers[1].get_weights()[0]:
                second_layer_w.append(np.flip(w_row).tolist())

            second_layer_b = np.flip(model_keras.layers[1].get_weights()[1])

            second_layer = [np.array(second_layer_w, dtype=np.float32),
                            np.array(second_layer_b, dtype=np.float32)]
            adapted_nn.layers[2].set_weights(second_layer)

        adapted_nn.layers[3].set_weights(
            [np.array([[0.0], [1.0]], dtype=np.float32), np.array([0.0], dtype=np.float32)])

        if len(num_feats) > 0:
            df = pd.read_csv(f'./data/NORM_{dsName}.csv')
        else:
            df = pd.read_csv(f'./data/{dsName}.csv')

        if len(cat_feats) > 0 and len(num_feats) > 0:
            df_oh = pd.read_csv(f'./data/OH_NORM_{dsName}.csv')
        elif len(cat_feats) > 0:
            df_oh = pd.read_csv(f'./data/OH_{dsName}.csv')
        else:
            df_oh = pd.read_csv(f'./data/NORM_{dsName}.csv')

        df_cf = pd.read_csv(f'./cfoutput/{n}_{dsName}_{framework}.csv')
        df_train = pd.read_csv(f'./experiments_data/{dsName}_TRAINDATASET.csv')
        df_fc = pd.read_csv(f'./experiments_data/{dsName}_CFDATASET_{n}.csv').drop(columns=['Unnamed: 0'])

        converter = OHConverter.Converter(df, cat_feats, list(df_oh.columns))

        print(n, dsName, framework)

        scoreValidityTotal = validity_total(df_cf, df_fc, adapted_nn)
        scoreValidityFound = validity_found(df_cf, df_fc, adapted_nn)
        scoreSparsity = sparsity(df_cf, df_fc, cat_feats, adapted_nn, converter)
        scoreL2 = l2(df_cf, df_fc, cat_feats, adapted_nn, converter)
        scoreMadd = madd(df_oh, df_cf, df_fc, cat_feats, num_feats, adapted_nn, converter)
        scoreMd = md(df_oh, df_cf, df_fc, cat_feats, num_feats, adapted_nn, converter)

        df_cf_found, df_fc_found = get_cleaned_processed(df_cf, df_fc, cat_feats, adapted_nn, converter)

        ruc = []
        rmc = []

        scoreCheckBinCat = check_binary_categorical(df_oh, df_cf, df_fc, cat_feats, num_feats, adapted_nn, converter)
        if scoreCheckBinCat is not np.nan:
            ruc.append(scoreCheckBinCat)

        scoreCheckOutsideNumRange = check_outside_num_range(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, adapted_nn,
                                                            converter)
        if scoreCheckOutsideNumRange is not np.nan:
            ruc.append(scoreCheckOutsideNumRange)

        scoreCheckOneHotIntegrity = check_one_hot_integrity(df_oh, df_cf, df_fc, cat_feats, num_feats, adapted_nn, converter)
        if scoreCheckOneHotIntegrity is not np.nan:
            rmc.append(scoreCheckOneHotIntegrity)

        if dsName in list(dict_functions.keys()):
            custom_ruc_functions = dict_functions[dsName]['ruc']
            custom_rmc_functions = dict_functions[dsName]['rmc']

            for c_ruc_function in custom_ruc_functions:
                output_ruc = c_ruc_function(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, adapted_nn, converter)
                if output_ruc is not np.nan:
                    ruc.append(output_ruc)
                else:
                    assert len(ruc) == 0

            for c_rmc_function in custom_rmc_functions:
                output_rmc = c_rmc_function(dsName, df_oh, df_cf, df_fc, cat_feats, num_feats, adapted_nn, converter)
                if output_rmc is not np.nan:
                    rmc.append(output_rmc)
                else:
                    assert len(rmc) == 0

        if len(ruc) > 0:
            scoreRUC = (sum(ruc) / len(ruc) == 1).map(int).tolist()

            out_array = scoreRUC
            output_results = [np.nan] * df_cf.shape[0]
            for idx_result, idxFound in enumerate(list(df_fc_found.index)):
                output_results[idxFound] = out_array[idx_result]

            scoreRUC = output_results

        else:
            scoreRUC = [np.nan]*df_cf.shape[0]

        if len(rmc) > 0:
            scoreRMC = (sum(rmc) / len(rmc) == 1).map(int).tolist()

            out_array = scoreRMC
            output_results = [np.nan] * df_cf.shape[0]
            for idx_result, idxFound in enumerate(list(df_fc_found.index)):
                output_results[idxFound] = out_array[idx_result]

            scoreRMC = output_results

        else:
            scoreRMC = [np.nan]*df_cf.shape[0]

        if dsName not in dict_results.keys():
            dict_results[dsName] = {}

        if n not in dict_results[dsName].keys():
            dict_results[dsName][n] = []

        result_entry = {
            'framework': framework,
            'n': n,
            'validity': scoreValidityTotal,
            'validityFound': scoreValidityFound,
            'sparsity': scoreSparsity,
            'L2': scoreL2,
            'RUC': scoreRUC,
            'RMC': scoreRMC,
            'MAD': scoreMadd,
            'MD': scoreMd
        }

        dict_results[dsName][n].append(result_entry)

1 StatlogGC SEDC


/home/rmazzine/anaconda3/envs/ALIBIC/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


1 SoybeanSmall GROWINGSPHERES4
1 PBC GROWINGSPHERES4


/home/rmazzine/anaconda3/envs/ALIBIC/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)
/home/rmazzine/anaconda3/envs/ALIBIC/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


1 SDD LORE


...

/home/rmazzine/anaconda3/envs/ALIBIC/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


0 Chess MLEXPLAIN
1 Nursery MACE
0 StudentPerf CADEX
0 StudentPerf MLEXPLAIN
0 TicTacToe LORE
1 DefaultOfCCC ALIBIC


/home/rmazzine/anaconda3/envs/ALIBIC/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


0 Lenses ALIBICNOGRAD
0 HayesRoth SYNAS
1 SDD CADEX


/home/rmazzine/anaconda3/envs/ALIBIC/lib/python3.7/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.1 when using version 0.23.2. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [32]:
cvnt001 = { 2: 2.576, 3: 2.913, 4: 3.113, 5: 3.255, 6: 3.364, 7: 3.452, 8: 3.526, 9: 3.590, 10: 3.646, 11: 3.696,
            12: 3.741, 13: 3.781, 14: 3.818, 15: 3.853, 16: 3.884, 17: 3.914, 18: 3.941, 19: 3.967, 20: 3.992,
            21: 4.015, 22: 4.037, 23: 4.057, 24: 4.077, 25: 4.096, 26: 4.114, 27: 4.132, 28: 4.148, 29: 4.164,
            30: 4.179, 31: 4.194, 32: 4.208, 33: 4.222, 34: 4.236, 35: 4.249, 36: 4.261, 37: 4.273, 38: 4.285,
            39: 4.296, 40: 4.307, 41: 4.318, 42: 4.329, 43: 4.339, 44: 4.349, 45: 4.359, 46: 4.368, 47: 4.378,
            48: 4.387, 49: 4.395, 50: 4.404, }

cvnt005 = { 2: 1.960, 3: 2.344, 4: 2.569, 5: 2.728, 6: 2.850, 7: 2.948, 8: 3.031, 9: 3.102, 10: 3.164, 11: 3.219,
            12: 3.268, 13: 3.313, 14: 3.354, 15: 3.391, 16: 3.426, 17: 3.458, 18: 3.489, 19: 3.517, 20: 3.544,
            21: 3.569, 22: 3.593, 23: 3.616, 24: 3.637, 25: 3.658, 26: 3.678, 27: 3.696, 28: 3.714, 29: 3.732,
            30: 3.749, 31: 3.765, 32: 3.780, 33: 3.795, 34: 3.810, 35: 3.824, 36: 3.837, 37: 3.850, 38: 3.863,
            39: 3.876, 40: 3.888, 41: 3.899, 42: 3.911, 43: 3.922, 44: 3.933, 45: 3.943, 46: 3.954, 47: 3.964,
            48: 3.973, 49: 3.983, 50: 3.992, }

cvnt010 = { 2: 1.645, 3: 2.052, 4: 2.291, 5: 2.460, 6: 2.589, 7: 2.693, 8: 2.780, 9: 2.855, 10: 2.920, 11: 2.978,
            12: 3.030, 13: 3.077, 14: 3.120, 15: 3.159, 16: 3.196, 17: 3.230, 18: 3.261, 19: 3.291, 20: 3.319,
            21: 3.346, 22: 3.371, 23: 3.394, 24: 3.417, 25: 3.439, 26: 3.459, 27: 3.479, 28: 3.498, 29: 3.516,
            30: 3.533, 31: 3.550, 32: 3.567, 33: 3.582, 34: 3.597, 35: 3.612, 36: 3.626, 37: 3.640, 38: 3.653,
            39: 3.666, 40: 3.679, 41: 3.691, 42: 3.703, 43: 3.714, 44: 3.726, 45: 3.737, 46: 3.747, 47: 3.758,
            48: 3.768, 49: 3.778, 50: 3.788, }

In [59]:
framework_names = ['ALIBIC', 'ALIBICNOGRAD', 'CADEX', 'DICE', 'GROWINGSPHERES3', 
                   'GROWINGSPHERES4', 'LORE', 'MACE', 'MLEXPLAIN', 'SEDC', 'SYNAS']

In [60]:
dsNames = np.sort(list(VAR_TYPES.keys())).tolist()

In [162]:
scores_parameters = {
    'validity': 'max',
    'sparsity': 'max',
    'L2': 'min',
    'RUC': 'max',
    'RMC': 'max',
    'MAD': 'min',
    'MD': 'min'
}

In [163]:
def calculate_rankings(row, d):
    best_values = row.dropna().unique()

    if d == 'max':
        best_values[::-1].sort()
    if d == 'min':
        best_values.sort()

    ranking_row = {}

    counter_score = 1
    for bv in best_values:
        sfw = list(row[row == bv].index)
        score = sum([*range(counter_score, counter_score + len(sfw))])/len(sfw)
        counter_score += len(sfw)
        for fw in sfw:
            ranking_row[fw] = score

    nan_fws = list(set(framework_names) - set(ranking_row.keys()))
    if len(nan_fws) > 0:
        nan_score = sum([*range(counter_score, counter_score + len(nan_fws))]) / len(nan_fws)

        for fw in nan_fws:
            ranking_row[fw] = nan_score

    return ranking_row

In [164]:
def get_mean_ranking(dsName, n, parameter, d):
    data_results = dict_results[dsName][n]
    total_parameter = []
    names_frameworks = []
    number_of_rows = -1
    for data in data_results:
        names_frameworks.append(data['framework'])
        total_parameter.append(data[parameter])
        if number_of_rows == -1:
            number_of_rows = len(data[parameter])
        else:
            assert len(data[parameter]) == number_of_rows

    df_param = pd.DataFrame(total_parameter).T
    df_param.columns = names_frameworks

    ranking_all_rows = []
    for idx, row in df_param.iterrows():

        ranking_row = calculate_rankings(row, d)

        ranking_all_rows.append(ranking_row)

    return ranking_all_rows

In [165]:
result_rows_best = []
for dsName in dict_results.keys():
    for n in dict_results[dsName].keys():
        # Just to get the frameworks sorted
        frameworks_sorted = np.sort(framework_names).tolist()

        for score_name, score_order in scores_parameters.items():
            result_best = get_mean_ranking(dsName, n, score_name, score_order)
            Ndataset = len(result_best)
            df_best = pd.DataFrame(result_best)[frameworks_sorted]
            df_best = df_best.mean().to_frame(0).T
            df_best['N'] = Ndataset
            
            df_best['dataset'] = dsName
            df_best['n'] = n
            df_best['score'] = score_name

            df_best.index = [f'{dsName}_{n}_{score_name}_{score_order}']

            result_rows_best.append(df_best)

df_mean_ranking = pd.concat(result_rows_best)

In [166]:
df_mean_ranking.loc[list(df_mean_ranking.index)]

,ALIBIC,ALIBICNOGRAD,CADEX,DICE,GROWINGSPHERES3,GROWINGSPHERES4,LORE,MACE,MLEXPLAIN,SEDC,SYNAS,N,dataset,n,score
StatlogGC_1_validity_max,8.580,4.01,4.010,4.010,4.080,4.010,8.98,10.97,8.98,4.150,4.220,100,StatlogGC,1,validity
StatlogGC_1_sparsity_max,9.085,2.77,6.975,5.775,5.295,3.500,9.51,9.51,9.51,2.765,1.305,100,StatlogGC,1,sparsity
StatlogGC_1_L2_min,9.130,4.09,6.620,5.270,1.070,2.410,9.51,9.51,9.51,3.075,5.805,100,StatlogGC,1,L2
StatlogGC_1_RUC_max,8.980,3.54,4.175,4.175,4.230,4.140,9.51,9.51,9.51,4.140,4.090,100,StatlogGC,1,RUC
StatlogGC_1_RMC_max,8.960,2.32,2.320,5.365,5.875,5.685,9.51,9.51,9.51,4.415,2.530,100,StatlogGC,1,RMC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ISOLET_1_L2_min,6.410,6.27,1.020,7.190,7.190,2.180,7.19,7.19,7.19,6.980,7.190,100,ISOLET,1,L2
ISOLET_1_RUC_max,6.400,6.40,2.145,7.190,7.190,1.145,7.19,7.19,7.19,6.770,7.190,100,ISOLET,1,RUC
ISOLET_1_RMC_max,6.000,6.00,6.000,6.000,6.000,6.000,6.00,6.00,6.00,6.000,6.000,100,ISOLET,1,RMC
ISOLET_1_MAD_min,6.870,6.85,2.040,7.030,7.030,1.030,7.03,7.03,7.03,7.030,7.030,100,ISOLET,1,MAD


In [167]:
df_mean_ranking[
    (df_mean_ranking['dataset']=='SDD')&
    (df_mean_ranking['n']=='0')
].drop(columns=['N', 'dataset', 'n', 'score'])

,ALIBIC,ALIBICNOGRAD,CADEX,DICE,GROWINGSPHERES3,GROWINGSPHERES4,LORE,MACE,MLEXPLAIN,SEDC,SYNAS
SDD_0_validity_max,5.645,5.645,4.705,5.645,8.815,1.395,6.965,10.250,5.645,5.645,5.645
SDD_0_sparsity_max,6.895,6.895,3.880,6.895,5.880,1.150,6.825,6.895,6.895,6.895,6.895
SDD_0_L2_min,6.895,6.895,3.530,6.895,5.970,1.380,6.855,6.895,6.895,6.895,6.895
SDD_0_RUC_max,6.895,6.895,3.615,6.895,6.005,1.275,6.840,6.895,6.895,6.895,6.895
SDD_0_RMC_max,6.000,6.000,6.000,6.000,6.000,6.000,6.000,6.000,6.000,6.000,6.000
SDD_0_MAD_min,6.730,6.730,4.630,6.730,6.530,1.000,6.730,6.730,6.730,6.730,6.730
SDD_0_MD_min,6.895,6.895,3.240,6.895,5.870,1.770,6.855,6.895,6.895,6.895,6.895


In [168]:
best_fws = []
for name, df in df_mean_ranking.drop(columns=['dataset', 'n', 'score']).iterrows():

    # No of treatments = No of frameworks / No of tests = No of rows

    N = df['N']
    k = (df.shape[0]-1)

    spx2f = sum([(R-(k+1)/2)**2 for R in df.drop('N').values])

    x2f = 12*N/(k*(k+1))*spx2f

    ff = (N-1)*x2f/(N*(k-1)-x2f)

    # 95% of confidence
    critical_f_value = sp.stats.f.ppf(q=1-.05, dfn=k-1, dfd=N-1)
    
    reject_h0 = True if ff >= critical_f_value else False

    cd = cvnt005[k]*(k*(k+1)/(6*N))**0.5



    unique_ranks = df.drop('N').unique().tolist()

    best_result = min(unique_ranks)

    groups = []
    for rc in unique_ranks:
        rc_group = [rc]
        for rt in list(set(unique_ranks)-set([rc])):
            if abs(rt-rc) < cd:
                rc_group.append(rt)
        dupe = False
        for gp in groups:
            if set(gp) == set(rc_group):
                dupe = True
        if not dupe:

            groups.append(rc_group)

    best_group = []
    for gp in groups:
        if best_result in gp:
            best_group += gp

    best_fw = {}
    for fw, rank in df.drop('N').items():
        if rank in best_group:
            best_fw[fw] = 1
        else:
            best_fw[fw] = 0

    res_out = pd.DataFrame([best_fw])[framework_names]
    res_out.index = [name]
    
    res_out['H0_Reject'] = reject_h0
    
    res_out['dataset'] = name.split('_')[0]
    res_out['n'] = name.split('_')[1]
    res_out['score'] = name.split('_')[2]

    best_fws.append(res_out)

df_best_fws = pd.concat(best_fws)

In [169]:
def highlight_best(s):
    '''
    highlight the statistically significant with gray.
    '''
    
    has_best = df_best_fws.loc[s.name]['H0_Reject']
    
    if has_best:
        
        bests = df_best_fws.loc[s.name].drop(['H0_Reject', 'dataset', 'n', 'score'])
        bests = bests[bests==1]
        bests_fws = list(bests[bests==1].index)
        
        return ['background-color: gray' if v in bests_fws else '' for v in list(s.index)]
    is_max = s == s.max()
    
    return ['']*len(s)

In [173]:
df_mean_ranking[
    (df_mean_ranking['n']=='0')
].drop(columns=['N', 'dataset', 'n', 'score']).style.apply(highlight_max, axis=1).to_excel('analysis.xlsx')